In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/bangkok_traffy.csv')

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
#df.head()

In [ ]:
footpath_keywords = [
    "ทางเท้า","ฟุตบาท","พื้นผิว","ฝาท่อ","แผ่นพื้น",
    "พื้นแตก","แตก","รอยแตก","หลุม","บ่อ",
    "พื้นทางเดิน","กระเบื้อง","แผ่นทางเท้า"
]

pattern = "|".join(footpath_keywords)

df_fp = df[
    df["type"].astype(str).str.contains(pattern, case=False, na=False) |
    df["comment"].astype(str).str.contains(pattern, case=False, na=False)
]

In [ ]:
df_fp.shape

In [ ]:
df_fp[["lon","lat"]] = df_fp["coords"].str.split(",", expand=True)
df_fp["lon"] = pd.to_numeric(df_fp["lon"], errors="coerce")
df_fp["lat"] = pd.to_numeric(df_fp["lat"], errors="coerce")

In [ ]:
df_fp["timestamp"] = pd.to_datetime(df_fp["timestamp"], errors="coerce")
df_fp["last_activity"] = pd.to_datetime(df_fp["last_activity"], errors="coerce")

In [ ]:
df_fp["duration_hours"] = (
    df_fp["last_activity"] - df_fp["timestamp"]
).dt.total_seconds() / 3600

df_fp["duration_hours"].describe()

In [ ]:
df_fp = df_fp[df_fp["duration_hours"] > 0]

df_fp = df_fp[df_fp["state"] =='เสร็จสิ้น']

lower_bound = df_fp["duration_hours"].quantile(0.01)
upper_bound = df_fp["duration_hours"].quantile(0.99)

df_fp["duration_hours"] = df_fp["duration_hours"].clip(lower=lower_bound, upper=upper_bound)
df_fp["duration_hours"].describe()

print(lower_bound)

In [ ]:
#df_fp.sort_values(by="duration_hours", ascending=True, inplace=True)
#df_fp.head()

In [ ]:
df_fp.shape

In [ ]:
output_path = "../data/processed/footpath_phase1.csv"
df_fp.to_csv(output_path, index=False)

len(df_fp), output_path